In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [18]:
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT * from Fires", conn)

In [3]:
# selected features from 'First irrelevant features selection.ipynb'
# also no need for 'STATE' feature - already in 'LATITUDE','LONGITUDE'

features = ['CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 
            'DISCOVERY_TIME', 'FIRE_SIZE', 'FIRE_YEAR', 'LATITUDE','LONGITUDE', 
            'NWCG_REPORTING_AGENCY', 'OWNER_DESCR','STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR']

### NWCG_REPORTING_AGENCY
will use as 5 dummies ST/C&L, FS, BIA, BLM and OTHER.

In [20]:
df.NWCG_REPORTING_AGENCY.unique()

array(['FS', 'BIA', 'TRIBE', 'BLM', 'NPS', 'BOR', 'FWS', 'ST/C&L', 'DOD',
       'IA', 'DOE'], dtype=object)

In [21]:
NWCG_REPORTING_AGENCY = df.NWCG_REPORTING_AGENCY.apply(lambda x: x if x in ["ST/C&L", "FS", "BIA", "BLM"] else "Other")
NWCG_REPORTING_AGENCY.value_counts()

ST/C&L    1377090
FS         220497
BIA        119943
BLM         97034
Other       65901
Name: NWCG_REPORTING_AGENCY, dtype: int64

In [22]:
df.NWCG_REPORTING_AGENCY = NWCG_REPORTING_AGENCY

## OWNER_DESCR
we will use 4 dummies: MISSING/NOT SPECIFIED, PRIVATE, USFS and OTHER

In [31]:
df.OWNER_DESCR.unique()

array(['USFS', 'STATE OR PRIVATE', 'MISSING/NOT SPECIFIED',
       'OTHER FEDERAL', 'BIA', 'FWS', 'TRIBAL', 'PRIVATE', 'STATE', 'BLM',
       'NPS', 'BOR', 'FOREIGN', 'MUNICIPAL/LOCAL', 'COUNTY',
       'UNDEFINED FEDERAL'], dtype=object)

In [32]:
OWNER_DESCR = df.OWNER_DESCR.apply(lambda x: x if x in ["MISSING/NOT SPECIFIED", "PRIVATE", "USFS"] else "Other")
OWNER_DESCR.value_counts()

MISSING/NOT SPECIFIED    1050835
Other                     326470
PRIVATE                   314822
USFS                      188338
Name: OWNER_DESCR, dtype: int64

In [33]:
df.OWNER_DESCR = OWNER_DESCR

# Time features

In [19]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')

df['DISCOVERY_MONTH'] = df['DISCOVERY_DATE'].dt.month
df['DISCOVERY_DAY'] = df['DISCOVERY_DATE'].dt.weekday

df['DISCOVERY_TIME'] = pd.to_datetime(df['DISCOVERY_TIME'], format='%H%M')
df['CONT_TIME'] = pd.to_datetime(df['CONT_TIME'], format='%H%M', errors='coerce')

## burn days

In [23]:
df['BURN_DAYS'] = (df['CONT_DATE'] - df['DISCOVERY_DATE']).astype('timedelta64[D]')

In [24]:
df['DISCOVERY_DATE'].head(2)

0   2005-02-02
1   2004-05-12
Name: DISCOVERY_DATE, dtype: datetime64[ns]

In [25]:
df['CONT_DATE'].head(2)

0   2005-02-02
1   2004-05-12
Name: CONT_DATE, dtype: datetime64[ns]

In [26]:
df['BURN_DAYS'].head(2) # will fill nulls according to train median

0    0.0
1    0.0
Name: BURN_DAYS, dtype: float64

## cyclic time

In [27]:
DAYS_IN_YEAR = 365
DAYS_IN_WEEK = 7

# no need for month cyclic - the data can be derived from DOY (day of year).

df['SIN_DISC_DOY'] = np.sin(2*np.pi*df['DISCOVERY_DOY']/DAYS_IN_YEAR)
df['COS_DISC_DOY'] = np.cos(2*np.pi*df['DISCOVERY_DOY']/DAYS_IN_YEAR)

df['SIN_DISC_WEEKDAY'] = np.sin(2*np.pi*df['DISCOVERY_DAY']/DAYS_IN_WEEK)
df['COS_DISC_WEEKDAY'] = np.cos(2*np.pi*df['DISCOVERY_DAY']/DAYS_IN_WEEK)

## weekend

In [29]:
df['IS_WEEKEND'] = (df['DISCOVERY_DAY'].isin([5, 6]))

## Independence day

In [30]:
df['INDEPENDENCE_DAY'] = ((df['DISCOVERY_DATE'].dt.day == 4) & (df['DISCOVERY_MONTH'] == 7))

# Elevation

In [34]:
elevation = pd.read_csv("elevation.csv")
elevation.head(1)

,Unnamed: 0,ELEVATION
0,0,924.73


In [35]:
elevation.ELEVATION.isnull().sum() # no null

0

In [36]:
elevation.drop("Unnamed: 0", axis=1, inplace=True)
elevation.head(1)

,ELEVATION
0,924.73


In [39]:
# add elevation data to df
df = pd.concat([df, elevation], axis=1)
df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,DISCOVERY_MONTH,DISCOVERY_DAY,BURN_DAYS,SIN_DISC_DOY,COS_DISC_DOY,SIN_DISC_WEEKDAY,COS_DISC_WEEKDAY,IS_WEEKEND,INDEPENDENCE_DAY,ELEVATION
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,2,2,0.0,0.538005,0.842942,0.974928,-0.222521,False,False,924.73
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,5,2,0.0,0.752667,-0.658402,0.974928,-0.222521,False,False,1831.26
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,5,0,0.0,0.501242,-0.865307,0.000000,1.000000,False,False,1048.65
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,6,0,5.0,0.043022,-0.999074,0.000000,1.000000,False,False,2361.86
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,6,0,5.0,0.043022,-0.999074,0.000000,1.000000,False,False,2309.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,1880461,300348363,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,...,9,5,0.0,-0.996659,-0.081676,-0.974928,-0.222521,True,False,214.46
1880461,1880462,300348373,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,10,0,NaN,-0.997325,0.073095,0.000000,1.000000,False,False,18.29
1880462,1880463,300348375,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,5,5,NaN,0.863142,-0.504961,-0.974928,-0.222521,True,False,18.29
1880463,1880464,300348377,2015CAIRS29218079,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,10,2,NaN,-0.974100,0.226116,0.974928,-0.222521,False,False,39.29


# Weather 

In [42]:
weather_df = pd.read_csv("weather_df.csv")
weather_df.head(1)

,Unnamed: 0,tavg 0 days before,tavg 1 days before,tmin 0 days before,tmin 1 days before,tmax 0 days before,tmax 1 days before,wspd 0 days before,wspd 1 days before,prcp 0 days before,prcp 1 days before,pres 0 days before,pres 1 days before
0,0,1.1,1.1,-2.2,-2.8,4.5,4.5,3.2,4.3,0.0,0.0,1035.9,1033.4


In [44]:
weather_df.drop("Unnamed: 0", axis=1, inplace=True)
weather_df.head(1)

,tavg 0 days before,tavg 1 days before,tmin 0 days before,tmin 1 days before,tmax 0 days before,tmax 1 days before,wspd 0 days before,wspd 1 days before,prcp 0 days before,prcp 1 days before,pres 0 days before,pres 1 days before
0,1.1,1.1,-2.2,-2.8,4.5,4.5,3.2,4.3,0.0,0.0,1035.9,1033.4


In [45]:
weather_df.isnull().sum() / weather_df.shape[0] # will fill nulls with train median

tavg 0 days before    0.013292
tavg 1 days before    0.012956
tmin 0 days before    0.004425
tmin 1 days before    0.004445
tmax 0 days before    0.004425
tmax 1 days before    0.004442
wspd 0 days before    0.012668
wspd 1 days before    0.012494
prcp 0 days before    0.037716
prcp 1 days before    0.037207
pres 0 days before    0.057530
pres 1 days before    0.058004
dtype: float64

In [46]:
# add weather_df data to df
df = pd.concat([df, weather_df], axis=1)
df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,tmin 0 days before,tmin 1 days before,tmax 0 days before,tmax 1 days before,wspd 0 days before,wspd 1 days before,prcp 0 days before,prcp 1 days before,pres 0 days before,pres 1 days before
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,-2.2,-2.8,4.5,4.5,3.2,4.3,0.0,0.0,1035.9,1033.4
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,-0.1,-0.6,9.9,8.3,7.3,10.9,0.0,5.5,1016.2,1012.4
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,17.1,14.9,24.9,23.2,12.6,5.7,0.0,0.0,1013.1,1016.1
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,8.1,7.6,6.4,5.3,6.0,7.5,0.0,0.0,1015.5,1015.5
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,8.4,7.9,6.7,5.6,6.0,7.5,0.0,0.0,1015.5,1015.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,1880461,300348363,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,...,14.0,12.4,34.6,35.2,4.0,5.8,0.0,0.0,1010.0,1010.5
1880461,1880462,300348373,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,13.4,10.7,26.8,25.1,7.3,11.2,0.0,0.0,1009.8,1001.9
1880462,1880463,300348375,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,16.8,16.2,32.9,35.1,9.0,10.4,0.0,0.0,1008.9,1009.1
1880463,1880464,300348377,2015CAIRS29218079,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,17.7,17.1,32.1,35.5,11.6,13.9,0.0,0.0,1013.5,1014.4


not relevant to preprocess: 'FIRE_SIZE', 'FIRE_YEAR', 'LATITUDE','LONGITUDE', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR'

# drop duplicates

In [54]:
df.shape

(1880465, 62)

In [55]:
# remove duplicates observations: same location, date and  STAT_CAUSE_CODE
no_dups = df.drop_duplicates(subset=["DISCOVERY_DATE", "LATITUDE", "LONGITUDE", "STAT_CAUSE_CODE"])

In [56]:
no_dups.shape

(1853533, 62)

In [64]:
# remove inconsistent observations: same location and  date but diffrent STAT_CAUSE_CODE
dup_check = no_dups[["DISCOVERY_DATE", "LATITUDE", "LONGITUDE", "STAT_CAUSE_CODE"]]

In [67]:
dup_check[dup_check[["DISCOVERY_DATE", "LATITUDE", "LONGITUDE"]].duplicated(keep=False)] # False : Mark all duplicates as True.

,DISCOVERY_DATE,LATITUDE,LONGITUDE,STAT_CAUSE_CODE
1026,2005-05-15,35.15,-111.64,9.00
1030,2005-05-15,35.15,-111.64,7.00
2590,2005-09-09,42.51,-122.41,4.00
2708,2005-09-09,42.51,-122.41,1.00
5030,2005-07-01,38.94,-120.05,4.00
...,...,...,...,...
1878530,2009-06-22,39.51,-121.35,6.00
1879193,2010-07-11,36.08,-118.90,7.00
1879709,2010-07-11,36.08,-118.90,9.00
1879880,2014-05-28,37.34,-119.66,9.00


In [69]:
# we can see 1026, 1030 both has same location and date but deffrent cause - remove them.
no_dups = no_dups[~dup_check[["DISCOVERY_DATE", "LATITUDE", "LONGITUDE"]].duplicated(keep=False)]
no_dups.shape 

(1847186, 62)

reduced from 1880465 to 1847186 <br>
run on the df

In [71]:
# remove duplicates observations: same location, date and  STAT_CAUSE_CODE
# df = df.drop_duplicates(subset=["DISCOVERY_DATE", "LATITUDE", "LONGITUDE", "STAT_CAUSE_CODE"])
dup_check = df[["DISCOVERY_DATE", "LATITUDE", "LONGITUDE", "STAT_CAUSE_CODE"]]
# remove inconsistent observations: same location and  date but diffrent STAT_CAUSE_CODE
df = df[~dup_check[["DISCOVERY_DATE", "LATITUDE", "LONGITUDE"]].duplicated(keep=False)]

In [72]:
df.shape

(1847186, 62)

In [73]:
# end of preprocess - save preprocessed df, to skip it next time
df.to_csv("preprocessed.csv")

# train_test_split_df function

In [74]:
from sklearn.model_selection import train_test_split

def train_test_split_df(df):
    target = "STAT_CAUSE_CODE"
    
    # too many nulls in burn days, to remove? 
    time_features = ["FIRE_YEAR", "SIN_DISC_DOY", "COS_DISC_DOY", "SIN_DISC_WEEKDAY",  
                     "COS_DISC_WEEKDAY", "IS_WEEKEND", "INDEPENDENCE_DAY", "BURN_DAYS"]
   
    weather_features = ["tavg 0 days before", "tavg 1 days before", "tmin 0 days before", "tmin 1 days before",
                        "tmax 0 days before", "tmax 1 days before", "wspd 0 days before", "wspd 1 days before", 
                        "prcp 0 days before", "prcp 1 days before", "pres 0 days before", "pres 1 days before" ]
   
    geographical_features = ['LATITUDE', 'LONGITUDE', 'ELEVATION']
    
    other_features = ["FIRE_SIZE", "NWCG_REPORTING_AGENCY", 'OWNER_DESCR']
    
    y = df[target]
    X = df[time_features + weather_features + geographical_features + other_features]
    
    # split with stratify - for the unbalanced target.
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, stratify=y)
    
    # get dummies 
    categorical_cols = ["NWCG_REPORTING_AGENCY", 'OWNER_DESCR']
    
    X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True) 
    X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True) 
    
    # match the dummies
    cols_in_both = sorted(set(X_train.columns).intersection(set(X_test.columns)))
    X_train = X_train[cols_in_both]
    X_test = X_test[cols_in_both]
    
    return X_train, X_test, y_train, y_test

In [75]:
X_train, X_test, y_train, y_test = train_test_split_df(df)

In [78]:
X_train.isnull().sum() / X_train.shape[0]

BURN_DAYS                      0.47
COS_DISC_DOY                   0.00
COS_DISC_WEEKDAY               0.00
ELEVATION                      0.00
FIRE_SIZE                      0.00
FIRE_YEAR                      0.00
INDEPENDENCE_DAY               0.00
IS_WEEKEND                     0.00
LATITUDE                       0.00
LONGITUDE                      0.00
NWCG_REPORTING_AGENCY_BLM      0.00
NWCG_REPORTING_AGENCY_FS       0.00
NWCG_REPORTING_AGENCY_Other    0.00
NWCG_REPORTING_AGENCY_ST/C&L   0.00
OWNER_DESCR_Other              0.00
OWNER_DESCR_PRIVATE            0.00
OWNER_DESCR_USFS               0.00
SIN_DISC_DOY                   0.00
SIN_DISC_WEEKDAY               0.00
prcp 0 days before             0.04
prcp 1 days before             0.04
pres 0 days before             0.06
pres 1 days before             0.06
tavg 0 days before             0.01
tavg 1 days before             0.01
tmax 0 days before             0.00
tmax 1 days before             0.00
tmin 0 days before          

In [77]:
X_test.isnull().sum() / X_test.shape[0]

BURN_DAYS                      0.47
COS_DISC_DOY                   0.00
COS_DISC_WEEKDAY               0.00
ELEVATION                      0.00
FIRE_SIZE                      0.00
FIRE_YEAR                      0.00
INDEPENDENCE_DAY               0.00
IS_WEEKEND                     0.00
LATITUDE                       0.00
LONGITUDE                      0.00
NWCG_REPORTING_AGENCY_BLM      0.00
NWCG_REPORTING_AGENCY_FS       0.00
NWCG_REPORTING_AGENCY_Other    0.00
NWCG_REPORTING_AGENCY_ST/C&L   0.00
OWNER_DESCR_Other              0.00
OWNER_DESCR_PRIVATE            0.00
OWNER_DESCR_USFS               0.00
SIN_DISC_DOY                   0.00
SIN_DISC_WEEKDAY               0.00
prcp 0 days before             0.04
prcp 1 days before             0.04
pres 0 days before             0.06
pres 1 days before             0.06
tavg 0 days before             0.01
tavg 1 days before             0.01
tmax 0 days before             0.00
tmax 1 days before             0.00
tmin 0 days before          

In [79]:
def fill_na_Train_Test(X_train, X_test):
    X_train_median = X_train.median()
    X_train = X_train.fillna(X_train_median)
    X_test = X_test.fillna(X_train_median)
    return X_train, X_test

In [80]:
X_train.median()

BURN_DAYS                         0.00
COS_DISC_DOY                     -0.28
COS_DISC_WEEKDAY                 -0.22
ELEVATION                       226.96
FIRE_SIZE                         1.00
FIRE_YEAR                      2004.00
INDEPENDENCE_DAY                  0.00
IS_WEEKEND                        0.00
LATITUDE                         35.48
LONGITUDE                       -92.14
NWCG_REPORTING_AGENCY_BLM         0.00
NWCG_REPORTING_AGENCY_FS          0.00
NWCG_REPORTING_AGENCY_Other       0.00
NWCG_REPORTING_AGENCY_ST/C&L      1.00
OWNER_DESCR_Other                 0.00
OWNER_DESCR_PRIVATE               0.00
OWNER_DESCR_USFS                  0.00
SIN_DISC_DOY                      0.21
SIN_DISC_WEEKDAY                  0.00
prcp 0 days before                0.00
prcp 1 days before                0.00
pres 0 days before             1016.00
pres 1 days before             1016.20
tavg 0 days before               17.20
tavg 1 days before               16.90
tmax 0 days before       

In [81]:
X_train, X_test = fill_na_Train_Test(X_train, X_test)

In [82]:
X_train.isnull().sum() 

BURN_DAYS                       0
COS_DISC_DOY                    0
COS_DISC_WEEKDAY                0
ELEVATION                       0
FIRE_SIZE                       0
FIRE_YEAR                       0
INDEPENDENCE_DAY                0
IS_WEEKEND                      0
LATITUDE                        0
LONGITUDE                       0
NWCG_REPORTING_AGENCY_BLM       0
NWCG_REPORTING_AGENCY_FS        0
NWCG_REPORTING_AGENCY_Other     0
NWCG_REPORTING_AGENCY_ST/C&L    0
OWNER_DESCR_Other               0
OWNER_DESCR_PRIVATE             0
OWNER_DESCR_USFS                0
SIN_DISC_DOY                    0
SIN_DISC_WEEKDAY                0
prcp 0 days before              0
prcp 1 days before              0
pres 0 days before              0
pres 1 days before              0
tavg 0 days before              0
tavg 1 days before              0
tmax 0 days before              0
tmax 1 days before              0
tmin 0 days before              0
tmin 1 days before              0
wspd 0 days be

In [83]:
X_test.isnull().sum() 

BURN_DAYS                       0
COS_DISC_DOY                    0
COS_DISC_WEEKDAY                0
ELEVATION                       0
FIRE_SIZE                       0
FIRE_YEAR                       0
INDEPENDENCE_DAY                0
IS_WEEKEND                      0
LATITUDE                        0
LONGITUDE                       0
NWCG_REPORTING_AGENCY_BLM       0
NWCG_REPORTING_AGENCY_FS        0
NWCG_REPORTING_AGENCY_Other     0
NWCG_REPORTING_AGENCY_ST/C&L    0
OWNER_DESCR_Other               0
OWNER_DESCR_PRIVATE             0
OWNER_DESCR_USFS                0
SIN_DISC_DOY                    0
SIN_DISC_WEEKDAY                0
prcp 0 days before              0
prcp 1 days before              0
pres 0 days before              0
pres 1 days before              0
tavg 0 days before              0
tavg 1 days before              0
tmax 0 days before              0
tmax 1 days before              0
tmin 0 days before              0
tmin 1 days before              0
wspd 0 days be

**no nulls**

# Choose model
failed with lazy predict, check between multiple which is the best, after choosing model will do hyper-parameters tuning for him.

In [2]:
def get_X_y(df):
    target = "STAT_CAUSE_CODE"

    # too many nulls in burn days, to remove?
    time_features = ["FIRE_YEAR", "SIN_DISC_DOY", "COS_DISC_DOY", "SIN_DISC_WEEKDAY", 
                     "COS_DISC_WEEKDAY", "IS_WEEKEND", "INDEPENDENCE_DAY", "BURN_DAYS"]

    weather_features = ["tavg 0 days before", "tavg 1 days before", "tmin 0 days before", "tmin 1 days before",
                        "tmax 0 days before", "tmax 1 days before", "wspd 0 days before", "wspd 1 days before", 
                        "prcp 0 days before", "prcp 1 days before", "pres 0 days before", "pres 1 days before" ]

    geographical_features = ['LATITUDE', 'LONGITUDE', 'ELEVATION']

    other_features = ["FIRE_SIZE", "NWCG_REPORTING_AGENCY", 'OWNER_DESCR']
    
    y = df[target]
    X = df[time_features + weather_features + geographical_features + other_features]
    # get dummies 
    categorical_cols = ["NWCG_REPORTING_AGENCY", 'OWNER_DESCR']
    X = pd.get_dummies(X, columns=categorical_cols, drop_first=True) 
    
    return X, y

In [37]:
target = ["STAT_CAUSE_CODE"]

time_features = ["FIRE_YEAR", "SIN_DISC_DOY", "COS_DISC_DOY", "SIN_DISC_WEEKDAY", 
                 "COS_DISC_WEEKDAY", "IS_WEEKEND", "INDEPENDENCE_DAY", "BURN_DAYS"]

weather_features = ["tavg 0 days before", "tavg 1 days before", "tmin 0 days before", "tmin 1 days before",
                    "tmax 0 days before", "tmax 1 days before", "wspd 0 days before", "wspd 1 days before", 
                    "prcp 0 days before", "prcp 1 days before", "pres 0 days before", "pres 1 days before" ]

geographical_features = ['LATITUDE', 'LONGITUDE', 'ELEVATION']

other_features = ["FIRE_SIZE", "NWCG_REPORTING_AGENCY", 'OWNER_DESCR']
features = time_features + weather_features + geographical_features + other_features + target

In [38]:
df = pd.read_csv("preprocessed.csv")[features]

/var/folders/7p/3bcxms1521j0nr_dz7tr2bjm0000gn/T/ipykernel_8300/1523457109.py:1: DtypeWarning: Columns (10,12,13,14,15,16,17,18,19,20,24,27,29,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("preprocessed.csv")[features]


In [39]:
df.head(2)

,FIRE_YEAR,SIN_DISC_DOY,COS_DISC_DOY,SIN_DISC_WEEKDAY,COS_DISC_WEEKDAY,IS_WEEKEND,INDEPENDENCE_DAY,BURN_DAYS,tavg 0 days before,tavg 1 days before,...,prcp 1 days before,pres 0 days before,pres 1 days before,LATITUDE,LONGITUDE,ELEVATION,FIRE_SIZE,NWCG_REPORTING_AGENCY,OWNER_DESCR,STAT_CAUSE_CODE
0,2005,0.538005,0.842942,0.974928,-0.222521,False,False,0.0,1.1,1.1,...,0.0,1035.9,1033.4,40.036944,-121.005833,924.73,0.10,FS,USFS,9.0
1,2004,0.752667,-0.658402,0.974928,-0.222521,False,False,0.0,5.6,3.9,...,5.5,1016.2,1012.4,38.933056,-120.404444,1831.26,0.25,FS,USFS,1.0


In [43]:
# drop null values before cross_val_score 
df_no_null = df.dropna()
df_no_null

,FIRE_YEAR,SIN_DISC_DOY,COS_DISC_DOY,SIN_DISC_WEEKDAY,COS_DISC_WEEKDAY,IS_WEEKEND,INDEPENDENCE_DAY,BURN_DAYS,tavg 0 days before,tavg 1 days before,...,prcp 1 days before,pres 0 days before,pres 1 days before,LATITUDE,LONGITUDE,ELEVATION,FIRE_SIZE,NWCG_REPORTING_AGENCY,OWNER_DESCR,STAT_CAUSE_CODE
0,2005,0.538005,0.842942,0.974928,-0.222521,False,False,0.0,1.1,1.1,...,0.0,1035.9,1033.4,40.036944,-121.005833,924.73,0.10,FS,USFS,9.0
1,2004,0.752667,-0.658402,0.974928,-0.222521,False,False,0.0,5.6,3.9,...,5.5,1016.2,1012.4,38.933056,-120.404444,1831.26,0.25,FS,USFS,1.0
2,2004,0.501242,-0.865307,0.000000,1.000000,False,False,0.0,21.1,18.8,...,0.0,1013.1,1016.1,38.984167,-120.735556,1048.65,0.10,FS,Other,5.0
3,2004,0.043022,-0.999074,0.000000,1.000000,False,False,5.0,11.7,11.1,...,0.0,1015.5,1015.5,38.559167,-119.913333,2361.86,0.10,FS,USFS,1.0
4,2004,0.043022,-0.999074,0.000000,1.000000,False,False,5.0,12.0,11.4,...,0.0,1015.5,1015.5,38.559167,-119.933056,2309.84,0.10,FS,USFS,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847177,2015,0.060213,-0.998186,-0.781831,0.623490,True,False,1.0,25.0,25.0,...,0.0,1012.8,1012.5,37.936253,-120.613743,188.39,5.30,ST/C&L,Other,9.0
1847178,2015,-0.999917,-0.012910,0.974928,-0.222521,False,False,1.0,12.4,15.4,...,0.0,1015.7,1013.6,40.588583,-123.069617,805.35,1.00,ST/C&L,Other,7.0
1847179,2015,-0.501242,-0.865307,-0.974928,-0.222521,True,False,5.0,26.1,25.8,...,0.0,1013.3,1011.8,40.244833,-123.544167,820.54,4.00,ST/C&L,Other,1.0
1847180,2015,0.559589,-0.828770,0.433884,-0.900969,False,False,0.0,14.5,14.1,...,0.0,1014.7,1014.7,38.415608,-122.660044,96.33,0.50,ST/C&L,Other,9.0


In [49]:
# sample stratified subset of the data to reduce cross_val_score running time.

# https://www.statology.org/stratified-sampling-pandas/
N = int(0.3 * df_no_null.shape[0])
samp = df_no_null.groupby('STAT_CAUSE_CODE', group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df_no_null))))).sample(frac=1).reset_index(drop=True)
samp

,FIRE_YEAR,SIN_DISC_DOY,COS_DISC_DOY,SIN_DISC_WEEKDAY,COS_DISC_WEEKDAY,IS_WEEKEND,INDEPENDENCE_DAY,BURN_DAYS,tavg 0 days before,tavg 1 days before,...,prcp 1 days before,pres 0 days before,pres 1 days before,LATITUDE,LONGITUDE,ELEVATION,FIRE_SIZE,NWCG_REPORTING_AGENCY,OWNER_DESCR,STAT_CAUSE_CODE
0,2007,0.025818,-0.999667,-0.974928,-0.222521,True,False,0.0,20.4,23.4,...,0.0,1017.1,1009.2,47.497200,-114.149000,922.04,0.10,BIA,PRIVATE,4.0
1,1994,0.280231,-0.959933,0.974928,-0.222521,False,False,0.0,6.5,6.3,...,0.0,1016.6,1016.5,43.096350,-122.983530,1046.28,0.10,ST/C&L,Other,2.0
2,2011,0.162807,-0.986658,0.974928,-0.222521,False,False,0.0,23.7,21.7,...,0.0,1010.7,1012.8,40.010000,-74.120000,0.42,0.25,ST/C&L,PRIVATE,1.0
3,2014,0.337523,0.941317,0.000000,1.000000,False,False,0.0,8.5,4.0,...,0.0,1014.5,1019.4,35.906670,-94.692830,350.56,2.00,ST/C&L,PRIVATE,7.0
4,2000,0.425000,-0.905193,0.000000,1.000000,False,False,0.0,22.2,22.7,...,0.0,1015.0,1015.6,34.440278,-118.589167,324.34,0.50,FS,MISSING/NOT SPECIFIED,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259936,1993,-0.961130,0.276097,-0.781831,0.623490,True,False,0.0,21.0,22.4,...,0.0,1009.6,1011.4,34.733853,-89.033436,122.79,1.00,ST/C&L,MISSING/NOT SPECIFIED,9.0
259937,1998,-0.179767,-0.983709,-0.781831,0.623490,True,False,1.0,21.3,17.9,...,0.0,1014.8,1019.7,44.950000,-88.650100,269.12,4.50,BIA,Other,4.0
259938,1997,0.705584,-0.708627,-0.974928,-0.222521,True,False,0.0,23.8,16.0,...,0.0,1013.3,1020.4,35.642500,-92.980000,373.00,5.00,FS,USFS,7.0
259939,2003,-0.769415,0.638749,0.000000,1.000000,False,False,0.0,0.0,-2.8,...,0.0,1032.3,1038.9,42.014570,-77.038233,453.35,1.50,ST/C&L,MISSING/NOT SPECIFIED,5.0


# Cross-Val on classifiers to pick the best between them.
Used random_state=1 to allow the results to be restored.

In [53]:
# import sklearn
#sklearn.metrics.SCORERS.keys()

**gradientBoostingClassifier:**

In [51]:
# https://machinelearningmastery.com/gradient-boosting-with-scikit-learn-xgboost-lightgbm-and-catboost/
## gradientBoostingClassifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

X, y = get_X_y(samp)

model = GradientBoostingClassifier(random_state=1)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [54]:
n_scores = cross_val_score(model, X, y, scoring='f1_weighted', cv=cv, n_jobs=-1, error_score='raise')

In [55]:
n_scores

array([0.54198913, 0.54344372, 0.54284654, 0.5410033 , 0.54018348,
       0.53964059, 0.54239428, 0.54274629, 0.5438884 , 0.54295226,
       0.54044885, 0.54203833, 0.54244022, 0.54487005, 0.54093497])

In [56]:
print('f1_weighted: mean=%.3f std=%.3f' % (mean(n_scores), std(n_scores)))

f1_weighted: mean=0.542 std=0.001


**XGBClassifier:**

In [61]:
from xgboost import XGBClassifier

In [72]:
X, y = get_X_y(samp)
y = y - 1 # for 'num_class' value in cv params
  
model = XGBClassifier(random_state=1)

In [69]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [73]:
n_scores = cross_val_score(model, X, y, scoring='f1_weighted', cv=cv, n_jobs=-1, error_score='raise')

In [74]:
n_scores

array([0.58558574, 0.58508504, 0.58748549, 0.58324008, 0.58418737,
       0.58473785, 0.5839543 , 0.5839077 , 0.58766981, 0.58489856,
       0.58542452, 0.58504425, 0.5845085 , 0.58539753, 0.58295078])

In [75]:
print('f1_weighted: mean=%.3f std=%.3f' % (mean(n_scores), std(n_scores)))

f1_weighted: mean=0.585 std=0.001


**RFClassifier:**

In [57]:
## RFClassifier

# https://machinelearningmastery.com/random-forest-ensemble-in-python/
# evaluate random forest algorithm for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
# define dataset
X, y =  get_X_y(samp)
X.fillna(X.median(), inplace=True)
# define the model
model = RandomForestClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [58]:
n_scores = cross_val_score(model, X, y, scoring='f1_weighted', cv=cv, n_jobs=-1, error_score='raise')

In [59]:
n_scores

array([0.55821226, 0.56034727, 0.56091535, 0.56013161, 0.56061009,
       0.55886019, 0.55961124, 0.56087614, 0.5602429 , 0.56136185,
       0.55894446, 0.56075505, 0.56006839, 0.56046063, 0.55969996])

In [60]:
# report performance
print('f1_weighted: mean=%.3f std=%.3f' % (mean(n_scores), std(n_scores)))

f1_weighted: mean=0.560 std=0.001


# Hyperparameter tuning 
RandomizedSearchCV

In [ ]:
# https://jamesrledoux.com/code/randomized_parameter_search

In [77]:
from scipy import stats
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

In [86]:
N = int(0.1 * df_no_null.shape[0])
samp = df_no_null.groupby('STAT_CAUSE_CODE', group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df_no_null))))).sample(frac=1).reset_index(drop=True)
samp

,FIRE_YEAR,SIN_DISC_DOY,COS_DISC_DOY,SIN_DISC_WEEKDAY,COS_DISC_WEEKDAY,IS_WEEKEND,INDEPENDENCE_DAY,BURN_DAYS,tavg 0 days before,tavg 1 days before,...,prcp 1 days before,pres 0 days before,pres 1 days before,LATITUDE,LONGITUDE,ELEVATION,FIRE_SIZE,NWCG_REPORTING_AGENCY,OWNER_DESCR,STAT_CAUSE_CODE
0,2012,0.717677,-0.696376,0.781831,0.623490,False,False,0.0,14.5,20.5,...,0.0,1018.7,1012.6,47.840800,-98.674200,448.10,1.00,BIA,Other,7.0
1,1999,-0.085965,0.996298,-0.781831,0.623490,True,False,0.0,6.8,5.1,...,0.0,1027.2,1031.1,31.166359,-88.716874,44.94,1.00,ST/C&L,MISSING/NOT SPECIFIED,7.0
2,2005,-0.903356,-0.428892,0.000000,1.000000,False,False,0.0,10.4,14.4,...,0.0,1018.0,1012.2,48.573600,-113.019000,1337.05,0.20,BIA,Other,9.0
3,1996,0.811539,0.584298,-0.974928,-0.222521,True,False,0.0,14.2,16.3,...,0.0,1017.6,1011.4,34.241500,-85.005500,217.33,0.94,ST/C&L,PRIVATE,5.0
4,1994,-0.025818,-0.999667,-0.781831,0.623490,True,False,0.0,14.2,16.5,...,0.0,1009.6,1003.7,65.599600,-148.502500,587.34,1.00,BLM,Other,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86640,2001,0.622047,0.782980,0.433884,-0.900969,False,False,0.0,10.6,8.9,...,0.0,1026.9,1024.4,33.298600,-84.280500,256.00,4.39,ST/C&L,PRIVATE,7.0
86641,1994,0.956235,0.292600,0.781831,0.623490,False,False,0.0,15.4,12.2,...,0.0,1010.3,1016.2,33.984400,-85.003100,297.51,3.75,ST/C&L,PRIVATE,6.0
86642,2014,0.448229,0.893919,0.000000,1.000000,False,False,0.0,7.7,7.2,...,0.0,1020.0,1019.2,38.398167,-122.207800,378.35,0.10,ST/C&L,MISSING/NOT SPECIFIED,2.0
86643,2002,-0.655156,-0.755493,0.000000,1.000000,False,False,0.0,22.8,20.2,...,0.0,1015.8,1016.4,43.655376,-75.960448,371.02,0.30,ST/C&L,MISSING/NOT SPECIFIED,4.0


In [87]:
X, y = get_X_y(samp)
y = y - 1 # for 'num_class' value in cv params

In [88]:
len(y.unique())

13

In [89]:
#https://stackoverflow.com/questions/43927725/python-hyperparameter-optimization-for-xgbclassifier-using-randomizedsearchcv

# multi:softmax: set XGBoost to do multiclass classification using the 
# softmax objective, you also need to set num_class(number of classes)

# Changed to: objective = 'multi:softmax'.
clf_xgb = XGBClassifier(objective = 'multi:softmax', num_class=len(y.unique()))

param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.59),
              'subsample': stats.uniform(0.3, 0.6),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.4),
              'min_child_weight': [1, 2, 3, 4]
             }

In [90]:
numFolds = 5
kfold_5 = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [93]:
clf = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = kfold_5,  
                         n_iter = 5, 
                         scoring = 'f1_weighted', 
                         error_score = 0, 
                         verbose = 1)

In [94]:
clf.fit(X, y)

Fitting 15 folds for each of 5 candidates, totalling 75 fits


RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
                   error_score=0,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importa...
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe3d3a921f0>,
                                        'learning_rate': <scipy.stats._di

In [95]:
clf.best_params_

{'colsample_bytree': 0.5787003004774066,
 'learning_rate': 0.07679335927606258,
 'max_depth': 4,
 'min_child_weight': 3,
 'n_estimators': 839,
 'subsample': 0.6642900020033061}

In [99]:
clf.best_score_

0.5674821813582024

# Train-Test on all data

In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv("preprocessed.csv")

/var/folders/7p/3bcxms1521j0nr_dz7tr2bjm0000gn/T/ipykernel_40861/340713475.py:1: DtypeWarning: Columns (10,12,13,14,15,16,17,18,19,20,24,27,29,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("preprocessed.csv")


In [47]:
def train_test_split_df(df):
    target = "STAT_CAUSE_CODE"
    
    time_features = ["FIRE_YEAR", "SIN_DISC_DOY", "COS_DISC_DOY", "SIN_DISC_WEEKDAY", 
                     "COS_DISC_WEEKDAY", "IS_WEEKEND", "INDEPENDENCE_DAY", "BURN_DAYS"]
   
    weather_features = ["tavg 0 days before", "tavg 1 days before", "tmin 0 days before", "tmin 1 days before",
                        "tmax 0 days before", "tmax 1 days before", "wspd 0 days before", "wspd 1 days before", 
                        "prcp 0 days before", "prcp 1 days before", "pres 0 days before", "pres 1 days before" ]
   
    geographical_features = ['LATITUDE', 'LONGITUDE', 'ELEVATION']
    
    other_features = ["FIRE_SIZE", "NWCG_REPORTING_AGENCY", 'OWNER_DESCR']
    
    y = df[target]
    X = df[time_features + weather_features + geographical_features + other_features]
    
    # split with stratify - becaues of the unbalanced target.
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, stratify=y)
    
    # get dummies 
    categorical_cols = ["NWCG_REPORTING_AGENCY", 'OWNER_DESCR']
    
    X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True) 
    X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True) 
    
    # match the dummies
    cols_in_both = sorted(set(X_train.columns).intersection(set(X_test.columns)))
    X_train = X_train[cols_in_both]
    X_test = X_test[cols_in_both]
    
    return X_train, X_test, y_train, y_test

def fill_na_Train_Test(X_train, X_test):
    X_train_median = X_train.median()
    X_train = X_train.fillna(X_train_median)
    X_test = X_test.fillna(X_train_median)
    return X_train, X_test

In [9]:
X_train, X_test, y_train, y_test = train_test_split_df(df)

y_train = y_train - 1
y_test = y_test - 1

X_train, X_test = fill_na_Train_Test(X_train, X_test)

In [12]:
XGB_best_params = {  'colsample_bytree': 0.5787,
                     'learning_rate': 0.0767,
                     'max_depth': 4,
                     'min_child_weight': 3,
                     'n_estimators': 839,
                     'subsample': 0.6643}

clf_xgb = XGBClassifier(objective = 'multi:softmax', num_class=13, **XGB_best_params)

In [16]:
clf_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5787,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.0767, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=3,
              missing=nan, monotone_constraints='()', n_estimators=839,
              n_jobs=0, num_class=13, num_parallel_tree=1,
              objective='multi:softmax', predictor='auto', random_state=0, ...)

In [17]:
pred = clf_xgb.predict(X_test)

In [52]:
from sklearn.metrics import f1_score

f1_score(y_test, pred, average=None)

array([0.80602873, 0.31497474, 0.06118814, 0.37962932, 0.59576746,
       0.47067784, 0.51804559, 0.17103763, 0.51000792, 0.50956987,
       0.12753359, 0.06077873, 0.88210459])

In [1]:
f1_score(y_test, pred, average='weighted')

0.5589388307510191